In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools

In [2]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [3]:
simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [4]:
ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')
ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [5]:
period = 'EV'
period_hr = 20

In [6]:
G = nx.read_gexf('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
graph_data = pd.read_csv('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')
stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [7]:
route_dir = stop_times_od[stop_times_od['hr_o'] == period_hr]
route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()
route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
route_dir = route_dir.merge(graph_data)

In [8]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0

In [9]:
result_list = []
ward_n = []

for ward in range(1,26,1):

    G_ward = G.copy()


    G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

    N = G_ward.size()

    #find cycles
    cycle_list = list(nx.simple_cycles(G_ward))
    print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

    temp_list = list(map(func, cycle_list))

    result_list.append(temp_list)
    ward_n.append([ward, N])


Ward 1, Size 342 Found 7256 cycles
Ward 2, Size 323 Found 263 cycles
Ward 3, Size 403 Found 324507 cycles
Ward 4, Size 210 Found 2037 cycles
Ward 5, Size 359 Found 63341 cycles
Ward 6, Size 268 Found 403 cycles
Ward 7, Size 228 Found 228 cycles
Ward 8, Size 267 Found 509 cycles
Ward 9, Size 187 Found 351 cycles
Ward 10, Size 118 Found 97 cycles
Ward 11, Size 175 Found 1918 cycles
Ward 12, Size 144 Found 70 cycles
Ward 13, Size 124 Found 118 cycles
Ward 14, Size 266 Found 7815 cycles
Ward 15, Size 248 Found 130 cycles
Ward 16, Size 190 Found 225 cycles
Ward 17, Size 204 Found 313 cycles
Ward 18, Size 206 Found 231 cycles
Ward 19, Size 237 Found 161 cycles
Ward 20, Size 321 Found 2143 cycles
Ward 21, Size 268 Found 704 cycles
Ward 22, Size 220 Found 9414 cycles
Ward 23, Size 262 Found 23839 cycles
Ward 24, Size 145 Found 73 cycles
Ward 25, Size 253 Found 123 cycles


In [10]:
combined = list(itertools.chain.from_iterable(result_list))
result_df = pd.DataFrame.from_records(
    combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
    'ward').sum().reset_index()
result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))
result_df

,ward,cycles,scaled_cycles,removed_cycles,N
0,1,7102,200.5375,154,342
1,2,108,2.6875,155,323
2,3,324325,8111.1000,182,403
3,4,1942,48.9375,95,210
4,5,63187,1438.5375,154,359
5,6,288,9.1875,115,268
6,7,141,6.6625,87,228
7,8,381,7.9125,128,267
8,9,263,8.2000,88,187
9,10,47,3.8875,50,118


In [11]:
result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)

In [12]:
result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                 right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
    columns = ['AREA_SHORT_CODE'])
result_df.sort_values(by = 'cycle_availability', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,ward_name
2,3,324325,8111.1000,182,403,10.001356,Etobicoke-Lakeshore
4,5,63187,1438.5375,154,359,1.989678,York South-Weston
22,23,23722,606.2250,117,262,1.145983,Scarborough North
21,22,9312,240.5875,102,220,0.540646,Scarborough-Agincourt
13,14,7690,199.8875,125,266,0.372230,Toronto-Danforth
0,1,7102,200.5375,154,342,0.291056,Etobicoke North
10,11,1840,48.1000,78,175,0.135493,University-Rosedale
3,4,1942,48.9375,95,210,0.115147,Parkdale-High Park
19,20,1998,52.4125,145,321,0.081009,Scarborough Southwest
20,21,582,28.3125,122,268,0.052334,Scarborough Centre


In [13]:
result_df.to_csv('cycles-' + period + '.csv', index = False)